## Unlearn finetuned model

In [40]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import Accelerator
from peft import PeftModel
import torch
import config
import args
import pickle
import utils
from utils import create_lotr_dataloader_from_dataset, get_answer_loss
from transformers import DataCollatorForLanguageModeling, get_scheduler
from torch.optim import AdamW
from accelerate import Accelerator
import importlib

In [39]:
importlib.reload(utils)

<module 'utils' from '/scratch/sa6981/llm_unlearn/finetune_copyright/utils.py'>

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [22]:
pretrained_model = AutoModelForCausalLM.from_pretrained(config.model_name, return_dict=True, load_in_8bit=True)
# pretrained_model.to(device)

tokenizer = AutoTokenizer.from_pretrained(config.model_name)
# # Load the Lora model
model = PeftModel.from_pretrained(pretrained_model, "/scratch/sa6981/llm_unlearn/finetune_copyright/models/finetune_1.3b")

In [26]:
optimizer = AdamW(model.parameters(), lr=args.lr)

In [27]:
num_training_steps = args.max_unlearn_steps
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [13]:
import pickle
with open('dataset_dict.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [18]:
# def create_lotr_dataloader_from_dataset(tokenizer, dataset, batch_size=4):
#     data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
#     dataloader = torch.utils.data.DataLoader(
#         dataset, batch_size=batch_size, collate_fn=data_collator
#     )
#     return dataloader

In [41]:
train_dataset = dataset["train"]
train_bad_loader = create_lotr_dataloader_from_dataset(
    tokenizer, train_dataset, batch_size=args.batch_size
    )

In [10]:
# Start unlearning.
bad_loss = 0.0
idx = 0
# start_time = time.time()

In [ ]:
# Stop if bad loss is big enough or reaching max step.
# start_time = time.time()
while bad_loss < args.max_bad_loss and idx < args.max_unlearn_steps:
    for bad_batch in (train_bad_loader):
        ############ GA on answer only. ############
        bad_loss = get_answer_loss("ga", bad_batch, model, device=device)

        ############ Random mismatch. ############
        random_loss = get_rand_ans_loss(
            bad_batch,
            tokenizer,
            normal_ans,
            model,
            K=5,
            device=device,
        )